In [336]:
from ipycanvas import Canvas

In [400]:
order = [('A','B','C','D','E','F','O'),
         ('a','b','c','d','e','f','o')]
letter_numbers = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5 }

## Klasse "Board"

In [352]:
class Board:
    
    def __init__(self):
        pass
    
    def _state_to_dict(self, state):
        d = {}
        for i in [0,1]:
            for j in range(7):
                d[order[i][j]] = state[i][j]
        return d
    
    def draw(self, state):
        troughes = self._state_to_dict(state)
        
        canvas = Canvas(width=800, height=350)

        canvas.fill_style='brown'
        canvas.fill_rect(0,0,canvas.width)


        canvas.stroke_circles([((i)*100)+140 for i in range(6)],[75 for i in range(6)], 40)
        canvas.stroke_circles([(i*100)+140 for i in range(6)],[255 for i in range(6)], 40)

        canvas.stroke_rects([10,canvas.width-100],[85,85],[80,80],[180,180])

        canvas.fill_style='white'
        canvas.font = '32px serif'
        canvas.fill_text("O",40,70)
        canvas.fill_text("F",135,30)
        canvas.fill_text("E",230,30)
        canvas.fill_text("D",330,30)
        canvas.fill_text("C",430,30)
        canvas.fill_text("B",530,30)
        canvas.fill_text("A",630,30)
        # ---------
        canvas.fill_style = 'black'
        canvas.fill_text("a",135,320)
        canvas.fill_text("b",230,320)
        canvas.fill_text("c",330,320)
        canvas.fill_text("d",430,320)
        canvas.fill_text("e",530,320)
        canvas.fill_text("f",630,320)
        canvas.fill_text("o",730,290)

        canvas.fill_style = 'black'
        canvas.fill_text(str(troughes['F']),135,140)
        canvas.fill_text(str(troughes['E']),230,140)

        canvas.fill_text(str(troughes['D']),330,140)
        canvas.fill_text(str(troughes['C']),430,140)
        canvas.fill_text(str(troughes['B']),530,140)
        canvas.fill_text(str(troughes['A']),630,140)

        canvas.fill_text(str(troughes['a']),135,210)
        canvas.fill_text(str(troughes['b']),230,210)
        canvas.fill_text(str(troughes['c']),330,210)
        canvas.fill_text(str(troughes['d']),430,210)
        canvas.fill_text(str(troughes['e']),530,210)
        canvas.fill_text(str(troughes['f']),630,210)

        canvas.fill_text(str(troughes['o']),730,175)
        canvas.fill_text(str(troughes['O']),40,175)

        return canvas

# Klasse "Player"

In [387]:
class Player:
    
    def __init__(self, number, name):
        if number in [0,1]:
            self.number = number
        else:
            raise ValueError("Number of player must be 0 or 1!")
        if isinstance(name, str):
            self.name = name
        else:
            raise ValueError("Name must be a string!")
    
    # own troughes should always be first, 
    # but in game state troughes of player 0 are first
    # -> if this is player 1: switch tuples in state
    def _switch_view(self, state):
        t1,t2 = state
        if self.number == 1:
            return [t2,t1]
        else:
            return [t1,t2]
        
    def _available_troughes(self, own_troughes):
        available = []
        for i in range(len(own_troughes)):
            if own_troughes[i] != 0:
                available.append(i)
        return available
        
    def choose_trough(self, current_state):
        pass

## Tests der Klasse "Player"

In [371]:
Player(2, "Player1")

ValueError: Number of player must be 0 or 1!

In [372]:
Player(0, 3)

ValueError: Name must be a string!

In [373]:
Player(0, "Player1")

## Player-Klasse "Human"

In [405]:
class Human(Player):
    
    def choose_trough(self, current_state):
        current_state = self._switch_view(current_state)
        own_t,safe = current_state[0][:6],current_state[0][6]
        available = self._available_troughes(own_t)
        
        i_string = "Choose one of the available troughes:\n"
        for i in available:
            i_string += f"{order[self.number][i]}, "
        
        choice_str = ""
        while choice_str not in letter_numbers.keys():
            choice_str = input(i_string[:-2]+"\n").lower()
        
        choice = letter_numbers[choice_str]
        return choice

### Tests der Player-Klasse "Human"

In [410]:
human = Human(0,"Hans")

In [412]:
human.choose_trough([(6,6,6,6,6,6,0), (6,6,6,6,6,6,0)])

Choose one of the available troughes:
A, B, C, D, E, F
B


1

## Player-Klasse "Random_KI"

In [450]:
import random as rn

class Random_KI(Player):
    
    rn.seed(1)
    
    def choose_trough(self, current_state):
        current_state = self._switch_view(current_state)
        own_t,safe = current_state[0][:6],current_state[0][6]
        available = self._available_troughes(own_t)
        
        choice = rn.choice(available)
        
        return choice

### Tests der Player-Klasse "Random_KI"

In [451]:
ki = Random_KI(0,"Rando")

In [454]:
ki.choose_trough([(6,6,6,6,6,6,0), (6,6,6,6,6,6,0)])

0

# Klasse "Kalah_Game"

In [359]:
class Kalah_Game():
    
    state = [(6,6,6,6,6,6,0), (6,6,6,6,6,6,0)]
    board = Board()
    
    def __init__(self, player):
        if len(player) != 2:
            raise ValueError("There must be exactly two players!")
        if not (isinstance(player[0], Player) and isinstance(player[1], Player)):
            raise ValueError("Both players must be of type Player!")
        self.player = player
        
    def show_states(self):
        s = f''
        for i in [0,1]:
            s += f'{self.player[i].name}:\t\t'
            for j in range(7):
                s += f'{order[i][j]}: {self.state[i][j]}  '
            s += f'\n'
        print(s)
    
    def draw_board(self):
        return self.board.draw(self.state)
    
    def _next_state(state, choice)
    
    def start(self):
        
        # only 2 turns right now
        for turn in range(1):
            

## Tests der Klasse "Kalah_Game":

In [360]:
game = Kalah_Game([])

ValueError: There must be exactly two players!

In [361]:
game = Kalah_Game([1, 2])

ValueError: Both players must be of type Player!

In [362]:
game = Kalah_Game([Random_KI(0,"KI"), Human(1,"Insa")])

ValueError: Both players must be of type Player!

In [363]:
game.draw_board()

Canvas(height=350, width=800)

In [323]:
game.show_states()

Insa:		A: 6  B: 6  C: 6  D: 6  E: 6  F: 6  O: 0  
KI:		a: 6  b: 6  c: 6  d: 6  e: 6  f: 6  o: 0  

